In [21]:
# The code was removed by DSX for sharing.

[Row(UTSUM_Electricity_Usage ;INFO_Year of Construction ;INFO_Number of Stories ;INFO_Total Square Feet ;PLEI_1_Quantity ;PLEI_3_Quantity=u'117'),
 Row(UTSUM_Electricity_Usage ;INFO_Year of Construction ;INFO_Number of Stories ;INFO_Total Square Feet ;PLEI_1_Quantity ;PLEI_3_Quantity=u'16'),
 Row(UTSUM_Electricity_Usage ;INFO_Year of Construction ;INFO_Number of Stories ;INFO_Total Square Feet ;PLEI_1_Quantity ;PLEI_3_Quantity=u'15'),
 Row(UTSUM_Electricity_Usage ;INFO_Year of Construction ;INFO_Number of Stories ;INFO_Total Square Feet ;PLEI_1_Quantity ;PLEI_3_Quantity=u'25'),
 Row(UTSUM_Electricity_Usage ;INFO_Year of Construction ;INFO_Number of Stories ;INFO_Total Square Feet ;PLEI_1_Quantity ;PLEI_3_Quantity=u'32')]

In [22]:
dfHDD = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(bmos.url('Exploring', 'HDD-Features.csv'))
dfHDD.take(5)

[Row(domestic_gas ;heating_gas=u'0.096226455 ;0.366193236'),
 Row(domestic_gas ;heating_gas=u'0.322599638 ;0.57959223'),
 Row(domestic_gas ;heating_gas=u'0.032705972 ;0.036460695'),
 Row(domestic_gas ;heating_gas=u'0.02750427 ;0.23466382'),
 Row(domestic_gas ;heating_gas=u'0.322599638 ;0.57959223')]

In [23]:
dfCH = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(bmos.url('Exploring', 'CDD-HDD-Features.csv'))
dfCH.take(5)

[Row(Property Name ;plug_load_consumption ;ac_consumption ;domestic_gas ;heating_gas=u'ChurchofStCeciliaReport ;1.165.140.596;0.983531348 ;0.096226455 ;0.366193236'),
 Row(Property Name ;plug_load_consumption ;ac_consumption ;domestic_gas ;heating_gas=u'69thLaneStudio ;3.276.988.438;5.008.371.873;0.322599638 ;0.57959223'),
 Row(Property Name ;plug_load_consumption ;ac_consumption ;domestic_gas ;heating_gas=u'UnitarianChurchofStatenIsland ;2.345.049.272;0.296133819 ;0.032705972 ;0.036460695'),
 Row(Property Name ;plug_load_consumption ;ac_consumption ;domestic_gas ;heating_gas=u'SSolowayandSonsPIPPrinting ;4.618.817.159;0.765188561 ;0.02750427 ;0.23466382'),
 Row(Property Name ;plug_load_consumption ;ac_consumption ;domestic_gas ;heating_gas=u'SunnysideJewishCenterReport ;9.323.896.186;123.432.624;0.322599638 ;0.57959223')]

In [24]:
import numpy as np
import pandas as pd
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

In [ ]:
df2.show(5)

In [26]:
# define cleaning functions
def energy(v): # reformat the values to get an actual number (e.g., 117,870 kWh to 117870)
    if (v=='') or v==None: return np.nan
    v = v.encode('ascii','ignore').split(' ')[0].replace(',','')
    return np.nan if(v=='') else float(v)
def age(v): # computes the age of a buildings, given the year of construction
    if (v=='') or v==None: return np.nan
    v = v.encode('ascii','ignore')
    return 2016.0-float(v) if(len(v)==4) else np.nan
def stories(v):
    if (v=='') or v==None: return np.nan
    return float(v)
def sqFeet(v): # get rid of commas 
    if (v=='') or v==None: return np.nan
    v = v.encode('ascii','ignore').replace(',','')
    return np.nan if(v=='') else float(v) 
def plei(v): # in the PLEI columns, missing values can be interpeted as 0 plugged equipment
    try:
        vv = float(v)
    except:
        vv = 0.0
    return vv 
# Define udf's to apply the defined function to the Spark DataFrame
udfEnergy = udf(energy, DoubleType())
udfAge = udf(age, DoubleType())
udfStories = udf(stories, DoubleType())
udfSqFeet = udf(sqFeet, DoubleType())
udfPlei = udf(plei, DoubleType())

In [27]:
dfN = df2.withColumn("UTSUM_Electricity_Usage", udfEnergy("UTSUM_Electricity_Usage")) \
         .withColumn("INFO_Year of Construction", udfAge("INFO_Year of Construction")) \
         .withColumn("INFO_Number of Stories", udfStories("INFO_Number of Stories")) \
         .withColumn("INFO_Total Square Feet", udfSqFeet("INFO_Total Square Feet")) \
         .withColumn("PLEI_1_Quantity", udfPlei("PLEI_1_Quantity")) \
         .withColumn("PLEI_3_Quantity", udfPlei("PLEI_3_Quantity")).cache()
dfN = dfN.withColumnRenamed("UTSUM_Electricity_Usage","energy") \
           .withColumnRenamed("INFO_Year of Construction","age") \
           .withColumnRenamed("INFO_Number of Stories","number_stories") \
           .withColumnRenamed("INFO_Total Square Feet","square_feet") \
           .withColumnRenamed("PLEI_1_Quantity","plei_1") \
           .withColumnRenamed("PLEI_3_Quantity","plei_3") 

AnalysisException: u"cannot resolve '`UTSUM_Electricity_Usage`' given input columns: [UTSUM_Electricity_Usage ;INFO_Year of Construction ;INFO_Number of Stories ;INFO_Total Square Feet ;PLEI_1_Quantity ;PLEI_3_Quantity];;\n'Project [UTSUM_Electricity_Usage ;INFO_Year of Construction ;INFO_Number of Stories ;INFO_Total Square Feet ;PLEI_1_Quantity ;PLEI_3_Quantity#41, energy('UTSUM_Electricity_Usage) AS UTSUM_Electricity_Usage#60]\n+- Relation[UTSUM_Electricity_Usage ;INFO_Year of Construction ;INFO_Number of Stories ;INFO_Total Square Feet ;PLEI_1_Quantity ;PLEI_3_Quantity#41] csv\n"

In [ ]:
dfN.take(1)

In [ ]:
# compute average of non-missing energy and age
energy_mean = np.nanmean(np.asarray(dfN.select("energy").rdd.map(lambda r: r[0]).collect()))
age_mean = np.nanmean(np.asarray(dfN.select("age").rdd.map(lambda r: r[0]).collect()))
# fill missing values with the computed average
dfN = dfN.na.fill({"energy": energy_mean, "age": age_mean})

In [ ]:
# define Spark DataFrame to be written to our object store
dfOut = dfN.select('energy', 'age', 'number_stories','square_feet','plei_1','plei_3')

In [ ]:
# use the .toPandas() function to map Spark DataFrames to pandas DataFrames
dfNp = dfN.toPandas()
dfHDDp = dfHDD.toPandas()
# concatenate two pandas DataFrames
feat = pd.concat([dfNp, dfHDDp], axis=1)
# get the column names of the concatenated DataFrame
cols = feat.columns
# scale data to prepare for regression model 
from sklearn import preprocessing
scaler = preprocessing.MaxAbsScaler() 
feat = scaler.fit_transform(feat)
# define a new DataFrame with the scaled data
dfScaled = pd.DataFrame(feat,columns=cols)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')
ff = pd.plotting.scatter_matrix(dfScaled, diagonal='hist',figsize=(12,12))

Fit a linear regression model to the observed data

Linear regression attempts to model the relationship between two variables by fitting a linear equation to the observed data. The energy usage (kWh) in buildings can be explained by considering the following building characteristics:

    Age of the building
    Square feet
    Number of stories
    Total number of plugged equipment

The regression modeler takes those building characteristics, which you prepared and scaled in an earlier step, and calculates the association between the proposed predicted value and the observed energy usage of the building.

In [ ]:
# get a list of the features used to explain energy
features = dfScaled.columns.tolist()
response = ['energy']
features.remove(response[0])
# import regression solver
from sklearn import linear_model
# declare a linear regression model 
lr = linear_model.LinearRegression(fit_intercept=True)
# define response variable: energy usage
y = np.asarray(dfScaled[response]) 
# define features
X = dfScaled[features]
# fit regression model to the data
regr = lr.fit(X,y)
coefs = regr.coef_[0]
# collect regression coefficients
dataRegQ = []
dataRegQ.append(('Intercept', regr.intercept_[0]))
for i in range(len(features)):
    dataRegQ.append((features[i],coefs[i]))
# compute energy predictions using our fitted model     
yh = regr.predict(X)
# import package to compute the R-squared quality metric
from sklearn.metrics import r2_score
# print results
print 'R-Squared: ', r2_score(y,yh)
pd.DataFrame(dataRegQ,columns=['feature_name','coefficient']) #.head()